In [2]:
from splink.duckdb.linker import DuckDBLinker
import pandas as pd
from splink.splink_dataframe import SplinkDataFrame
from typing import TYPE_CHECKING

# https://stackoverflow.com/questions/39740632/python-type-hinting-without-cyclic-imports
if TYPE_CHECKING:
    from .linker import Linker

In [3]:
from splink.datasets import splink_datasets
from splink.duckdb.blocking_rule_library import block_on, exact_match_rule
from splink.duckdb.comparison_library import (
    exact_match,
    levenshtein_at_thresholds,
)
from splink.duckdb.linker import DuckDBLinker

import ssl

ssl._create_default_https_context = ssl._create_unverified_context

df = splink_datasets.fake_1000

settings = {
    "probability_two_random_records_match": 0.01,
    "link_type": "dedupe_only",
    "blocking_rules_to_generate_predictions": [
        block_on(["first_name"]),
        exact_match_rule("surname"),
    ],
    "comparisons": [
        levenshtein_at_thresholds("first_name", 2),
        exact_match("surname"),
        exact_match("dob"),
        exact_match("city", term_frequency_adjustments=True),
        exact_match("email"),
    ],
    "retain_intermediate_calculation_columns": True,
    "additional_columns_to_retain": ["cluster"],
    "max_iterations": 10,
    "em_convergence": 0.01,
}


linker = DuckDBLinker(df, settings)

linker.estimate_u_using_random_sampling(target_rows=1e6)


blocking_rule = "l.first_name = r.first_name and l.surname = r.surname"
linker.estimate_parameters_using_expectation_maximisation(blocking_rule)


blocking_rule = "l.dob = r.dob"
linker.estimate_parameters_using_expectation_maximisation(blocking_rule)


df_predict = linker.predict()
df_clustered = linker.cluster_pairwise_predictions_at_threshold(df_predict, 0.9)

/var/folders/nd/c3xr518x3txg5kcqp1h7zwc80000gp/T/ipykernel_9019/4096503936.py:20: DeprecationWarning: `exact_match_rule` is deprecated; use `block_on`
  exact_match_rule("surname"),
/var/folders/nd/c3xr518x3txg5kcqp1h7zwc80000gp/T/ipykernel_9019/4096503936.py:38: SplinkDeprecated: target_rows is deprecated; use max_pairs
  linker.estimate_u_using_random_sampling(target_rows=1e6)
----- Estimating u probabilities using random sampling -----

Estimated u probabilities using random sampling

Your model is not yet fully trained. Missing estimates for:
    - first_name (no m values are trained).
    - surname (no m values are trained).
    - dob (no m values are trained).
    - city (no m values are trained).
    - email (no m values are trained).

----- Starting EM training session -----

Estimating the m probabilities of the model by blocking on:
l.first_name = r.first_name and l.surname = r.surname

Parameter estimates will be made for the following comparison(s):
    - dob
    - city
   

In [14]:
df_graph_metrics

GraphMetricsResults(nodes=<splink.duckdb.linker.DuckDBDataFrame object at 0x7fc99705d400>, edges=None, clusters=<splink.duckdb.linker.DuckDBDataFrame object at 0x7fc9970511f0>)

In [11]:
node_metrics = df_graph_metrics.nodes
edge_metrics = df_graph_metrics.edges
cluster_metrics = df_graph_metrics.clusters
cluster_metrics

Table name in database: `__splink__graph_metrics_clusters_d32d3c41d`

To retrieve records, you can call the following methods on this object:
`.as_record_dict(limit=5)` or `.as_pandas_dataframe(limit=5)`.

You may omit the `limit` argument to return all records.

This table represents the following splink entity: __splink__graph_metrics_clusters

In [ ]:
import matplotlib.pyplot as plt

metric = "node_degree"
metric_type = node_metrics_pd

plt.hist(node_metrics_pd[metric], bins=10)
plt.ylabel(metric)
plt.xlabel("count of clusters")


plt.show()